[![open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nagaoka-ai-innovationhub/basics-of-image-recognition-with-cnn/blob/master/07_basics_of_convolutional_neural_networks.ipynb)

# 畳み込みニューラルネットワークの基本

`tf.keras`とFashon MNISTを使って、畳み込みニューラルネットワークを構築してみましょう。

In [ ]:
# ColabでのTensorFlow 2.xのインストール
try:
    # %tensorflow_version は Colab 上でのみ使えます
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

# TensorFlowとtf.kerasのバージョン確認
print(tf.version.VERSION)
print(tf.keras.__version__)

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

# Fashon MNISTデータセットの読み込み
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# クラス名の定義
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 畳み込みニューラルネットワークモデルの構築

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))   # 3x3の畳み込みカーネル(フィルタ) を32個、入力層は(28 x 28 x 1チャンネル)
model.add(layers.MaxPooling2D((2, 2)))   # 2x2のMax Pooling
model.add(layers.Conv2D(64, (3, 3), activation='relu'))    # 3x3の畳み込みカーネル(フィルタ) を64個
model.add(layers.MaxPooling2D((2, 2)))   # 2x2のMax Pooling

model.add(layers.Flatten())     # 平滑化
model.add(layers.Dense(128, activation='relu'))            # ノード数が128個の全結合層
model.add(layers.Dense(10, activation='softmax'))     # 出力層

畳み込み層は`Conv2D`、プーリング層は`MaxPooling2D`を使用しています。
以下の部分が画像の畳み込み処理を行う層になり、画像の特徴を自動的に抽出しています。

```python
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
```

画像データ入力の際は`(28*28)`に平滑化せずに、`(height, width, channels)`で与えてあげます。`channels`はカラー画像であればRGBの3チャンネルを使うことが多いですが、今回は白黒のグレースケールなので1チャンネルにしています。

今回は畳み込み層1つ、プーリング層1つ、畳み込み層1つ、プーリング層1つの構成にしています。
各層の出力は全て`(height, width, channels)`になります。
畳込みカーネルの数だけ、チャンネルが増えます。また、プーリング層によって`(height, width)`は半分になります。

その後、畳み込み処理部分で得られた画像の特徴を元に、`Dense`層でニューラルネットワークと同じように分類をしています。

```python
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
```

`Dense`層へ値を渡す際には、パラメータの数分のベクトルで渡す必要があるので、その処理を`Flatten`で行なっています。
その後の処理はニューラルネットワークと同じ内容です。

モデルの構造を見てみましょう。

In [ ]:
model.summary()

`Flatten`層にて、`(5, 5, 64)`の出力は`(5*5*64)`に平滑化されていることがわかります。

In [ ]:
# オプション: 図でモデル構造を表示（graphviz本体のインストールをした後で、pipでpydotplusとgraphvizのインストールが必要)
#!pip install pydotplus
#!pip install graphviz
#from tensorflow.keras.utils import plot_model
#
#plot_model(model, to_file='basic_cnn_model.png')

コンパイルの内容はニューラルネットワークと同じです。

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',   # 損失関数は「categorical_crossentropy」
    optimizer = 'rmsprop',  # オプティマイザは「rmsprop」
    metrics = ['accuracy']  # 指標は「accuracy」（正答率）
)

## データの前処理

データの前処理をしましょう。今回は以下のような処理になります。
異なる点は、入力時に平滑化をする必要は無くなりましたが、チャネル数の部分を追加してあげる必要があります。

In [ ]:
from tensorflow.keras.utils import to_categorical

train_images = train_images.reshape((60000, 28, 28, 1))  # 入力配列の構造にチャネル部分を追加
train_images = train_images.astype('float32') / 255      # 値の範囲を[0,1]に正規化
train_labels = to_categorical(train_labels)                        # ラベルをone-hot encoding

test_images = test_images.reshape((10000, 28, 28, 1))    # 入力配列の構造にチャネル部分を追加
test_images = test_images.astype('float32') / 255        # 値の範囲を[0,1]に正規化
test_labels = to_categorical(test_labels)                          # ラベルをone-hot encoding

## モデルの学習

畳み込み部分が追加されているため、パラメータ数が非常に多くなりました。
そのため、学習には時間がかかります。

In [ ]:
# epochs: 全データを繰り返し学習する回数
# batch_size: 1度に投入する入力データ数
history = model.fit(train_images, train_labels, epochs=5, batch_size=128)

## 学習の推移をグラフ化

In [ ]:
# 学習の推移をグラフ化
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['accuracy']  # 学習データの正答率
loss = history.history['loss']  # 学習データの損失

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.legend()
plt.show()

## モデルの評価

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
print('test_acc: ', test_acc)

最後に上記コードをまとめておきます。

In [ ]:
# ColabでのTensorFlow 2.xのインストール
try:
    # %tensorflow_version は Colab 上でのみ使えます
    %tensorflow_version 2.x
except Exception:
    pass

# ライブラリの読み込み
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

# TensorFlowとtf.kerasのバージョン確認
print(tf.version.VERSION)
print(tf.keras.__version__)

# Fashon MNISTデータセットの読み込み
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# クラス名の定義
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# モデルの構築
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))   # 3x3の畳み込みカーネル(フィルタ) を32個、入力層は(28 x 28 x 1チャンネル)
model.add(layers.MaxPooling2D((2, 2)))   # 2x2のMax Pooling
model.add(layers.Conv2D(64, (3, 3), activation='relu'))    # 3x3の畳み込みカーネル(フィルタ) を64個
model.add(layers.MaxPooling2D((2, 2)))   # 2x2のMax Pooling

model.add(layers.Flatten())     # 平滑化
model.add(layers.Dense(128, activation='relu'))            # ノード数が128個の全結合層
model.add(layers.Dense(10, activation='softmax'))     # 出力層

# モデルのコンパイル
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'rmsprop',
    metrics = ['accuracy']
)

# 学習データの前処理
train_images = train_images.reshape((60000, 28, 28, 1))  # 入力配列の構造にチャネル部分を追加
train_images = train_images.astype('float32') / 255      # 値の範囲を[0,1]に正規化
train_labels = to_categorical(train_labels)                        # ラベルをone-hot encoding

test_images = test_images.reshape((10000, 28, 28, 1))    # 入力配列の構造にチャネル部分を追加
test_images = test_images.astype('float32') / 255        # 値の範囲を[0,1]に正規化
test_labels = to_categorical(test_labels)                          # ラベルをone-hot encoding

# モデルの学習
history = model.fit(train_images, train_labels, epochs = 5, batch_size = 128)

# 学習の推移をグラフ化
acc = history.history['accuracy']  # 学習データの正答率
loss = history.history['loss']  # 学習データの損失
epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.legend()
plt.show()

# 学習済モデルの評価
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test_acc: ', test_acc)

## 畳み込みニューラルネットワークは何を見ているか

ここでは、畳み込みニューラルネットワークについての理解を深めるために、学習済みのニューラルネットワークがどのような特徴をとらえているかを、隠れ層の出力（活性化マップ）を可視化することで見てみたいと思います。

まず、学習済みモデルのフラット化の前の4層の出力を取得し、tf.keras のモデル activation_model を新たに作成します、。

In [ ]:
layer_outputs = [layer.output for layer in model.layers[:4]]

# 既存モデルの入力と、上記で取得した各層の出力を出力とするモデルを構築
activation_model = models.Model(inputs=model.inputs, outputs=layer_outputs)

test_images の最初の画像を入力として、推論（順方向処理）を実行します。　　
test_images[0]とすると次元（階数）が1つ少ない (28, 28, 1)　の配列が得られるので、次元拡張によって形状を合わせていることに注意。

In [ ]:
import numpy as np

activations = activation_model.predict(np.expand_dims(test_images[0], 0))

activations は4層分あるので、最初の層の出力の形状をみてみましょう。

In [ ]:
print(activations[0].shape)

例として、インデックス3のフィルタの出力する特徴マップを、カラーマップを使って可視化します。

In [ ]:
plt.matshow(activations[0][0, :, :, 3], cmap='viridis')
plt.show()

各層のパラメータは乱数によって初期化されるため、上記の特徴マップは訓練を実行するごとに異なることに注意。

下記のプログラムにより、各層の特徴マップ全体を表示します。

In [ ]:
# 各レイヤの名前を抽出
layer_names = []
for layer in model.layers[:4]:
    layer_names.append(layer.name)

# 1行に表示するイメージ数
images_per_row = 16

# レイヤごとに処理を繰り返す
for layer_name, layer_activation in zip(layer_names, activations):
    # このレイヤの特徴マップの数を調べる
    n_features = layer_activation.shape[-1]

    # 特徴マップのサイズを取得（正方形を仮定している）
    size = layer_activation.shape[1]

    # 画像の行数を計算し画像を並べる0行列を生成
    n_rows = n_features // images_per_row
    display_grid = np.zeros((size * n_rows, images_per_row * size))

    # チャネルごとの特徴マップを取り出し行列に埋め込む
    for row in range(n_rows):
        for col in range(images_per_row):
            channel_image = layer_activation[0, :, :, row * images_per_row + col]
            # 特徴マップを平均0分散1に標準化
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            # 平均128標準偏差64に変換
            channel_image *= 64
            channel_image += 128
            # 0から255の範囲にクリッピングし符号なし8ビット整数化
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            # 所定の位置にイメージを埋め込む
            display_grid[row * size : (row + 1) * size,
                         col * size : (col + 1) * size] = channel_image

    # 画像の表示
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show()